# Получение вопроса, поиск нужных чанков, генерация ответа

### загрузка модели из data

In [1]:
# создаём путь до места хранения
from pathlib import Path

# Получаем текущий рабочий каталог
current_dir = Path.cwd()

# Создаем путь к папке processed_data, которая находится на уровне выше текущего каталога
processed_data_path = current_dir.parent / 'data' / 'processed_data' / 'union.parquet'


In [2]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings

vectorstore = SKLearnVectorStore(
    embedding=OllamaEmbeddings(model="llama3.1"), 
    persist_path=processed_data_path, 
    serializer="parquet"
)

retriever = vectorstore.as_retriever(k=3)


### Вопрос, поиск ответа

In [3]:
question = "Что делает оператор SELECT?"

### Развёртывание модели

In [4]:
from langchain.prompts import PromptTemplate

# Define the prompt template for the LLM
prompt = PromptTemplate(
    template="""Вы помощник в вопросах-ответах.
    Для ответа на вопрос используйте следующие документы если они полезны.
    Если вы не знаете ответа, поробуйте ответить самостоятельно или просто скажите, что не знаете.
    Вопрос {question}
    Документы: {documents}
    Отвечать:
    """,
    input_variables=["question", "documents"],
)

In [5]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model='llama3.1', temperature=0)


C:\Users\zenfo\AppData\Local\Temp\ipykernel_14884\1894568862.py:4: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model='llama3.1', temperature=0)


In [6]:
# Create a chain combining the prompt template and LLM
rag_chain = prompt | llm | StrOutputParser()

### Пердаём данные модели и получаем ответ

In [7]:
doc_texts = vectorstore.similarity_search(question)
doc_texts

[Document(metadata={'id': '790e0b5d-3feb-40ca-898e-5f2cca12383f', 'page': 2, 'source': '../data/standard_data/sql_primer.pdf'}, page_content='Показаны примеры использования транзакций, уделено внимание методам оптимизации запросов. Материал сопровождается многочисленными практическими примерами. Пособие может использоваться как для самостоятельного обучения, так и при проведении занятий под руководством преподавателя'),
 Document(metadata={'id': '7cc4eadb-1e8b-45e5-912c-914eb848bdad', 'page': 7, 'source': '../data/standard_data/sql_primer.pdf'}, page_content='Книга написана таким образом, чтобы ее можно было использовать как под руководством преподавателя, так и самостоятельно'),
 Document(metadata={'id': 'e7f6ece1-07b6-4b65-b531-af5350de9ebc', 'page': 5, 'source': '../data/standard_data/sql_primer.pdf'}, page_content='На эту роль идеально подходила СУБД PostgreSQL. Книга, которую вы держите в руках, появилась по инициативе и при поддержке компании Postgres Professional — российского п

In [9]:
answer = rag_chain.invoke({"question": question, "documents": doc_texts})

In [ ]:
# Вопрос
question

In [ ]:
# Что выдала модель со всеми данными
answer